# Decorators

데코레이터란 기능의 확장을 현제 가지고 있는 함수의 수정 없이 하는 기능 

- Function Decorator
- Class decorators

# Function decoators
- 데코레이터에 철학
- 파이썬은 first class object : 함수 내부와 외부는 다른 영역이다
- 데코레이터는 다른 함수를 argument로 만듬 

In [8]:
# A decorator function takes another function as argument, wraps its behaviour inside
# an inner function, and returns the wrapped function.
def start_end_decorator(func):
    
    def wrapper(): # wrapper란 다른 함수를 받아오는 부분
        print('Start')
        func()
        print('End')
    return wrapper

def print_name():
    print('Alex')
    
print_name()

print()

# Now wrap the function by passing it as argument to the decorator function
# and asign it to itself -> Our function has extended behaviour!
print_name = start_end_decorator(print_name)
print_name()

Alex

Start
Alex
End


# 데코레이터 신텍스
- wrapper를 쓰는 데신 선언을 먼저 함으로서 wrapper의 기능을 해준다

In [7]:
@start_end_decorator
def print_name():
    print('Alex')
    
print_name()

Start
End


# function arguments 
- 래핑을 할 떄 주의점
- 감싸고 싶은 함수가 input 파라미터가 있는 경우 wrapping하게 되면 
- 우리의 함수에 변수가 어떻게 들어올지 모름으로 typeerror를 뿜게 됨
- 그래서 우리는 *arg, **kwargs를 넣어준다

In [14]:
def start_end_decorator_2(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        func(*args, **kwargs)
        print('End')
    return wrapper

@start_end_decorator_2
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
None


# Return value

- 위에 예제에서 return을 정의 하지 않았음으로 print시 none이 나옴

In [19]:
def start_end_decorator_3(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_3
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
15


# function identity?

In [20]:
print(add_5.__name__)
help(add_5)

wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [21]:
def add_5(x):
    return x + 5

print(add_5.__name__)
help(add_5)

add_5
Help on function add_5 in module __main__:

add_5(x)



# 문제점 

- wrapper가 help에서 나오게 되고 안에 감싸져 있는 함수에 대한 정보는 없다
- functools.wraps 데코레이터를통해 안에 감싸져있는 원본 함수 정보를 보존해야함

In [24]:
import functools
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_4
def add_5(x):
    return x + 5
result = add_5(10)

print(result)
print(add_5.__name__)
help(add_5)

Start
End
15
add_5
Help on function add_5 in module __main__:

add_5(x)



# 데코레이터 template

In [25]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        result = func(*args, **kwargs)
        # Do something after
        return result
    return wrapper

# Decorator function arguments

In [26]:
def repeat(num_times):
    def decorator_repeat(func):
        
        @functools.wraps(func)
        
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")
    
greet('Alex')

Hello Alex
Hello Alex
Hello Alex


# Nested Decorators

In [29]:
# import functools
# def start_end_decorator_4(func):
    
#     @functools.wraps(func)
#     def wrapper(*args, **kwargs):
#         print('Start')
#         result = func(*args, **kwargs)
#         print('End')
#         return result
#     return wrapper

In [37]:
a = "Life is too short"
print(str(a)) # 직관적으로 사용자가 보기 쉽도록 
print(repr(a)) # 문자열로 객체를 다시 생성

Life is too short
'Life is too short'


In [39]:
import datetime

In [43]:
a = datetime.datetime(2017, 9, 27)
b = repr(a)
c = str(a)
eval(b)
print(b)
print(c)

datetime.datetime(2017, 9, 27, 0, 0)
2017-09-27 00:00:00


In [46]:
# a decorator function that prints debug information about the wrapped function
def debug(func):
    @functools.wraps(func)
    
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        print(args_repr)
        print(kwargs_repr)
        
        signature = ", ".join(args_repr + kwargs_repr)
        print(signature)
        
        print(f"Calling {func.__name__}({signature})")
        result = func(*args, **kwargs)
        
        print(f"{func.__name__!r} returned {result!r}")
        return result
    return wrapper

@debug
@start_end_decorator_4
def say_hello(name):
    greeting = f'Hello {name}'
    print(greeting)
    return greeting

# now `debug` is executed first and calls `@start_end_decorator_4`, which then calls `say_hello`
say_hello(name='Alex')

[]
["name='Alex'"]
name='Alex'
Calling say_hello(name='Alex')
Start
Hello Alex
End
'say_hello' returned 'Hello Alex'


'Hello Alex'

# Class decorators

- class에서도 데코레이터를 사용할수 있는데
- `__call__()` method 에서 사용 가능하다
- wrapper()와 동일한 기능을 한다

- `functools.update_wrapper()` 을 `functools.wraps` 사용해야 함수 정보를 살릴수 있음

In [50]:
import functools

class CountCalls:
    # the init needs to have the func as argument and stores it
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0
    
    # extend functionality, execute function, and return the result
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello(num):
    print("Hello!")
    
say_hello(5)
say_hello(5)

Call 1 of 'say_hello'
Hello!
Call 2 of 'say_hello'
Hello!


# decorator가 쓰이는 이유들
- time.sleep()을 쓸떄
- Cache return values for memorizaition
- Add information or update a state

In [1]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [2]:
print([fib(n) for n in range(16)])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]


In [53]:
print(fib.cache_info())

CacheInfo(hits=28, misses=16, maxsize=None, currsize=16)


In [3]:
%%timeit
([fib(n) for n in range(16)])

746 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
import functools

@functools.lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [7]:
import timeit
timeit.timeit('fibonacci(35)', globals=globals(), number=1)

5.49579999997718e-05

In [8]:
timeit.timeit('fibonacci(35)', globals=globals(), number=1)

3.5409999981084184e-06

In [9]:
from functools import cache

def fetch_user(user_id):
    print(f"DB에서 아이디가 {user_id}인 사용자 정보를 읽어오고 있습니다...")
    return {
        "userId": user_id,
        "email": f"{user_id}@test.com",
        "password": "test1234"
    }

@cache
def get_user(user_id):
    return fetch_user(user_id)


In [10]:
from random import choice

for i in range(10):
    get_user(user_id = choice(["A01", "B02", "C03"]))

DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 A01인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 C03인 사용자 정보를 읽어오고 있습니다...


In [14]:
def get_user(user_id):
    return fetch_user(user_id)

In [15]:
from random import choice

for i in range(10):
    get_user(user_id = choice(["A01", "B02", "C03"]))

DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 A01인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 C03인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 A01인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 C03인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 B02인 사용자 정보를 읽어오고 있습니다...
DB에서 아이디가 C03인 사용자 정보를 읽어오고 있습니다...


# 똒똒 하게 time.sleep 하기

In [1]:
import time
import urllib.request
import urllib.error

def sleep(timeout, retry=3):
    def the_real_decorator(function):
        def wrapper(*args, **kwargs):
            retries = 0
            while retries < retry:
                try:
                    value = function(*args, **kwargs)
                    if value is None:
                        return
                except:
                    print(f'Sleeping for {timeout} seconds')
                    time.sleep(timeout)
                    retries += 1
        return wrapper
    return the_real_decorator

In [2]:
@sleep(3)
def uptime_bot(url):
    try:
        conn = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e:
        # Email admin / log
        print(f'HTTPError: {e.code} for {url}')
        # Re-raise the exception for the decorator
        raise urllib.error.HTTPError
    except urllib.error.URLError as e:
        # Email admin / log
        print(f'URLError: {e.code} for {url}')
        # Re-raise the exception for the decorator
        raise urllib.error.URLError
    else:
        # Website is up
        print(f'{url} is up')

if __name__ == '__main__':
    url = 'http://www.google.com/py'
    uptime_bot(url)

HTTPError: 404 for http://www.google.com/py
Sleeping for 3 seconds
HTTPError: 404 for http://www.google.com/py
Sleeping for 3 seconds
HTTPError: 404 for http://www.google.com/py
Sleeping for 3 seconds


# machine learning

In [5]:
from sklearn import datasets
import pandas as pd
iris = datasets.load_iris(as_frame = True)

y = iris['target']
x = iris['data']

In [6]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

xtrain, xtest, ytrain, ytest = train_test_split(x, y)

rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
print(accuracy_score(ytest, y_pred))



rf = AdaBoostClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
print(accuracy_score(ytest, y_pred))


rf = ExtraTreesClassifier()
rf.fit(xtrain, ytrain)
y_pred = rf.predict(xtest)
print(accuracy_score(ytest, y_pred))


0.9473684210526315
0.9473684210526315
0.9473684210526315


In [35]:
from functools import wraps 

def model_registry(function): 
    @wraps(function)
    def wrapped(*args, **kwargs): 
        score, prediction, fmodel = function(*args, **kwargs)
        registry[args[0].__name__] = {'score': score, 'prediction': prediction} 
        print(f"{args[0].__name__} =  'score': {score}")
        return score, prediction, fmodel
    return wrapped

In [36]:
@model_registry
def fitter(model, xtrain, xtest, ytrain, ytest): 
    md = model()
    md.fit(xtrain, ytrain)
    y_pred = md.predict(xtest)
    return accuracy_score(ytest, y_pred), y_pred, md

In [37]:
registry = {}

score, prediction, fmodel = fitter(RandomForestClassifier, xtrain, xtest, ytrain, ytest)
score, prediction, fmodel = fitter(ExtraTreesClassifier, xtrain, xtest, ytrain, ytest)
score, prediction, fmodel = fitter(AdaBoostClassifier, xtrain, xtest, ytrain, ytest)

RandomForestClassifier =  'score': 0.9736842105263158
ExtraTreesClassifier =  'score': 0.9473684210526315
AdaBoostClassifier =  'score': 0.9473684210526315
